In [26]:
import vnstock as vn
import ta
import pandas as pd

In [27]:
import matplotlib.pyplot as plt

In [28]:
RSI_PERIOD = 14
RSI_OVERSOLD = 30
RSI_OVERBOUGHT = 70
MACD_FAST_PERIOD = 12
MACD_SLOW_PERIOD = 26
MACD_SIGNAL_PERIOD = 9

In [29]:
def calculate_indicators(df):
    if df.empty:
        return df
    
    df['RSI'] = ta.momentum.RSIIndicator(df['close'], RSI_PERIOD).rsi()
    df['Previous_RSI'] = df['RSI'].shift(1)
    df['Previous_RSI'].fillna(0, inplace=True)
    macd = ta.trend.MACD(df['close'], window_slow=MACD_SLOW_PERIOD, window_fast=MACD_FAST_PERIOD, window_sign=MACD_SIGNAL_PERIOD)
    df['MACD'] = macd.macd()
    df['Signal_Line'] = macd.macd_signal()
    df['Previous_MACD'] = df['MACD'].shift(1)
    df['Previous_Signal_Line'] = df['Signal_Line'].shift(1)
    df['Previous_MACD'].fillna(0, inplace=True)
    df['Previous_Signal_Line'].fillna(0, inplace=True)

    return df

In [30]:
def generate_signals(df):
    if df.empty:
        return df
    
    df['Signal'] = 0

    # Buy signals: RSI cross above 30 and MACD cross above Signal line
    df.loc[
        (df['Previous_MACD'] < df['Previous_Signal_Line']) &
           (df['MACD'] >= df['Signal_Line']) 
        #    & (df['Previous_RSI'] < RSI_OVERSOLD) 
           & (df['RSI'] > RSI_OVERSOLD), 'Signal'] = 1

    #Sell Signals: 
    df.loc[
        #    (  df['Previous_RSI'] > RSI_OVERBOUGHT) &
           (df['RSI'] < RSI_OVERBOUGHT) 
           & (df['Previous_MACD'] > df['Previous_Signal_Line']) 
           & (df['MACD'] <= df['Signal_Line']), 'Signal'] = -1

    return df

In [31]:
def backtest_strategy(df, initial_balance=160000000):
    if df.empty:
        return df
    
    balance = initial_balance
    shares = 0
    df['Portfolio Value'] = balance
    df['Shares'] = 0


    for i in range(1, len(df)):
        if df['Signal'].iloc[i] == 1:  # Buy signal
            if balance > 0: 
                shares_to_buy = balance // df['close'].iloc[i]
                balance -= shares_to_buy * df['close'].iloc[i]
                shares += shares_to_buy


        elif df['Signal'].iloc[i] == -1 and shares > 0:  # Sell signal
            balance += shares * df['close'].iloc[i]
            shares = 0

        df.loc[i, 'Portfolio Value'] = balance + shares * df['close'].iloc[i]
        df.loc[i, 'Shares'] = shares

    df.loc[len(df) - 1, 'Portfolio Value'] = balance + shares * df['close'].iloc[-1]

    return df

In [32]:
def visualize_results(df, symbol):
    plt.figure(figsize=(14, 10))
    
    plt.subplot(3, 1, 1)
    plt.plot(df.index, df['close'], label='Close Price')
    plt.plot(df.loc[df['Signal'] == 1].index, df['close'][df['Signal'] == 1], '^', markersize=10, color='g', lw=0, label='Buy Signal')
    plt.plot(df.loc[df['Signal'] == -1].index, df['close'][df['Signal'] == -1], 'v', markersize=10, color='r', lw=0, label='Sell Signal')
    plt.title(f'{symbol} Price Chart')
    plt.legend()
    
    plt.subplot(3, 1, 2)
    plt.plot(df.index, df['Portfolio Value'], label='Portfolio Value', color='b')
    plt.title(f'{symbol} Portfolio Value Over Time')
    plt.legend()

    plt.subplot(3, 1, 3)
    plt.plot(df.index, df['Shares'], label='Shares Held', color='purple')
    plt.title(f'{symbol} Shares Held Over Time')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

In [33]:
def calculate_profit_total(df, initial_balance=160000000):
    if df.empty:
        return 0
    final_balance = df['Portfolio Value'].iloc[-1] 
    total_change = ((final_balance - initial_balance)/initial_balance) * 100
    return total_change

In [34]:
def test_company(symbol, start_date, end_date, resolution='1D', type='stock'):
    df = vn.stock_historical_data(symbol=symbol, 
                            start_date=start_date, 
                            end_date=end_date, resolution=resolution, type=type)
    if df.empty:
        return 0 
    df = calculate_indicators(df)
    df = generate_signals(df)
    df = backtest_strategy(df)
    # visualize_results(df, symbol)
    total_profit_change = calculate_profit_total(df, initial_balance=160000000)
        
    return total_profit_change

In [35]:
START_DATE = '2021-01-01'
END_DATE = '2024-01-01'

VN30_list = [
    'SSI', 'BCM','VHM','VIC','VRE','BVH','POW','GAS','ACB','BID',
'CTG','HDB','MBB','SSB','SHB','STB','TCB','TPB','VCB','VIB','VPB','HPG',
'GVR','MSN','VNM','SAB','VJC','MWG','PLX','FPT']

In [36]:
profit_all = {}

for company in VN30_list:
    try:
        company_data_all = test_company(company, start_date=START_DATE, end_date=END_DATE, resolution='1D', type='stock')
       
        # Update dictionaries
        profit_all[company] = company_data_all

    except Exception as e:
        print(f"Error occurred for {company}: {e}")

C:\Users\Admin\AppData\Local\Temp\ipykernel_22400\245259429.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Previous_RSI'].fillna(0, inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_22400\245259429.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, w

In [37]:
profit_all_df = pd.DataFrame.from_dict(profit_all, orient='index', columns=['Profit During all period']).reset_index()
final_result = profit_all_df.rename(columns={'index': 'Company'})
final_result.to_csv('result/rsi_macd.csv', index=False)
